**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/gym_plannable.git
!{sys.executable} -m pip install git+https://github.com/michalgregor/rl_tabular.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
%matplotlib inline
from rl_tabular import ActionValueTable, StateValueTable
from rl_tabular.maze_env_plots import (
    Plotter, plot_action_values, plot_state_values)
from rl_tabular import EpsGreedyPolicy, ReplayBuffer, QLearning
from rl_tabular import ExponentialSchedule
from rl_tabular import qtable_control
from rl_tabular import Trainer, seed
from gym_plannable.env import MazeEnv
import matplotlib.pyplot as plt
import numpy as np

## Q učenie

Ďalej sa budeme venovať TD (temporal difference) prístupu známemu ako Q učenie. Q učenie je bezmodelové a učí sa optimálnu **hodnotovú funkci akcií** , takže model sa nevyžaduje ani na riadenie agenta (na rozdiel od TD učenia s hodnotovou funkciou stavov).

Napokon, Q učenie je metóda neviazaná na stratégiu (off-policy) takže sa dokáže učiť aj z dát nazbieraných pomocou inej stratégie – to nám umožní používať ho s opätovným prehrávaním skúseností.

### Q učenie s fixnou postupnosťou akcií

V prvom kroku sa budeme sústrediť na samotné Q učenie a nebudeme riešiť prieskum. Preto v nasledujúcej bunke definujeme fixnú postupnosť akcií, ktorá agenta dostane z počiatočného do cieľového stavu. Jedinou úlohou nášho agenta bude naučiť sa postupnosť akcií potom, ako ju pár ráz pozoruje. Uvidíme, ako dobre to bude fungovať.

---
### Úloha 1: Implementujte pravidlo Q učenia

**V nasledujúcej bunke doplňte implementáciu pravidla Q učenia.** 

$$
\begin{aligned}
\delta &= r_{t+1} + \gamma \underset{a}{\max} \ Q(s_{t+1}, a)-Q(s_{t},a_{t}) \\
Q(s_{t},a_{t}) &\leftarrow Q(s_{t},a_{t}) + \alpha \delta
\end{aligned}
$$
---


In [ ]:
actions = [0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 3]
alpha = 0.5
gamma = 0.9

# create the environment
env = MazeEnv() # we do not enable rendering here; we'll do it manually

# set up plotting
plotter = Plotter(env, ActionValueTable, StateValueTable, figsize=[8, 4])

# set up the value function
qtable = ActionValueTable(env.action_space.n)

# the training loop
step = 0
for episode in range(20):
    obs = env.reset()
        
    for a in actions:
        # apply the action and observe the effect
        obs_next, reward, done, _ = env.step(a)
        
        
        
        
        # compute td: a, reward, qtable, gamma, obs, obs_next, np.max
        td = # -----
        
        
        
        
        qtable[obs, a] += alpha * td
        
        # book keeping
        obs = obs_next
        step += 1

        # for the first 3 episodes, we also do step-wise plots
        if episode < 3: plotter.plot(qtable, qtable.to_state_values())

        # if the environment is done, conclude the episode
        if done: break

    print(f"Episode {episode} finished after {step} steps.")

In [ ]:
plt.figure()
plot_action_values(
    qtable, plotter.states, env=env, render_agent=False
)

plt.figure()
plot_state_values(
    qtable.to_state_values(), plotter.states, env=env, render_agent=False
)

Čo vidno z vizualizácie, je, že v prípade klasického Q učenia agent aktualizuje len hodnotu predposledného stavu. To dáva zmysel, pretože keď sa počítajú hodnoty predchádzajúcich stavov, ešte sa nevie, že agent získa za posledný stavový prechod odmenu.

Na druhej strane je zrejmé, že je to veľmi nepraktické – znamená to, že celú sekvenciu je potrebné zopakovať približne 20 ráz (približne dĺžka postupnosti) kým sa ju agent celú naučí – t.j. kým sa hodnota prešíri pozdĺž celej cesty z cieľového až do počiatočného stavu.

A aj to platí len za predpokladu, že agent bude každý raz pozorovať tú istú postupnosť akcií – pričom v bežných podmienkach by agent zakaždým preskúmaval inú cestu. Práve na tento aspekt prieskumu sa pozrieme teraz.

### Q učenie s $\varepsilon$-greedy prieskumom

V predošlom príklade sme používali v každej epizóde tú istú postupnosť akcií. Teraz budeme postupovať trochu realistickejším spôsobom: použijeme $\varepsilon$-greedy stratégiu s $\varepsilon = 0.1$, aby sme robili aj určitý prieskum.

Na tento experiment využijeme hotové implementácie stratégie aj Q učenia. Vizualizácie budú ukazovať oboje: hodnotovú funkciu akcií – kde **zafarbené štvorčeky**  vizualizujú **cestu, ktorú si agent zvolil**  v poslednej epizóde – a hodnotovú funkciu stavov. 

*Všimnite si riadok `seed(1)`. Tento fixuje jadro náhodného generátora tak, aby sme dostali tie isté výsledky pri každom spustení bunky. Ak chcete vidieť aj iné spôsoby, ako by učenie mohlo prebiehať, môžete tento riadok zakomentovať.* 



In [ ]:
seed(1)

env = MazeEnv(
    show_path=True,
    show_path_kw=dict(show_arrows=False, show_visited=True)
)

# set up plotting
plotter = Plotter(env, ActionValueTable, 
    (StateValueTable, {'render_kwargs': {'skip': {'player_logger'}}}),
     figsize=[8, 4], render_agent=False
)

qtable = ActionValueTable(env.action_space.n)
algo = QLearning(qtable, alpha=0.5, gamma=0.9)
policy = EpsGreedyPolicy(qtable, env.action_space.n, epsilon=0.1)
trainer = Trainer(
    algo, policy, verbose=5, on_end_episode=[
        lambda *args: plotter.plot(qtable, qtable.to_state_values())]
)

trainer.train(env, max_episodes=50, max_episode_steps=1000)

In [ ]:
env = MazeEnv(show_path=True)
qtable_control(env, qtable, render=False, max_steps=100)
env.render()

Ako vidno, agent sa pohybuje náhodne kým neobjaví cieľový stav – od tej chvíle sa začína učiť cestu, ktorá k nemu vedie. Vďaka $\varepsilon$-greedy stratégii stále robí aj trochu prieskumu: niekedy sa odchýli od toho, čo sa už naučil, vďaka čomu môže získať pre zmenu aj iné skúsenosti.

Platí však, že pri $\varepsilon = 0.1$ sa aj po veľmi veľkom počte epizód v skutočnosti naučí len jednu cestu do cieľového stavu a pozná len hodnoty stavov, ktoré ležia veľmi blízko nej.

---
### Úloha 2:

**Ako ďalšiu úlohu skúste experiment spustiť znovu s inými hodnotami $\varepsilon$. Sledujte čo sa mení keď sa $\varepsilon$ posúva bližšie k 1. Akú veľkú časť stavového priestoru agent preskúma? Aký je celkový počet krokov, ktoré agent vykoná v rámci predpísaných 50-tich epizód?** 

---


In [ ]:
env = MazeEnv(show_path=True)
qtable_control(env, qtable, max_steps=100)
env.render()

### Q učenie a opätovné prehrávanie skúseností

Ďalším problémom, ktorému sa musíme venovať, je nízka vzorková účinnosť nášho algoritmu. Spomeňte si, že sme tú istú cestu museli prejsť veľa krát, pretože agent si z nej každý raz zapamätal len jeden ďalší krok. Musí predsa existovať spôsob, ako sa naučiť celú cestu bez toho, aby ju agent musel vidieť viac než raz.

V skutočnosti existuje hneď niekoľko spôsobov, ako to docieliť – my budeme používať techniku známu ako opätovné prehrávanie skúseností (experience replay), kde si všetko, čo agent pozoroval, zaznamenáme a potom si časti týchto skúseností budeme viac ráz opätovne prehrávať. Tým spôsobom budeme vlastne vedieť prešíriť hodnotu prechodu do cieľového stavu späť ku počiatočnému stavu aj ak sme celú postupnosť akcií videli len raz.

#### Prehrávacia pamäť (replay buffer)

V našom prípade nebude treba prehrávaciu pamäť celú implementovať – mamé takú implementáciu už pripravenú. Pozrieme sa však teraz na jej rozhranie. Pri konštrukcii pamäte špecifikujeme jej maximálnu veľkosť (`max_size`). Keď sa dosiahne kapacita pamäte, nové skúsenosti začnú v pamäti nahrádzať tie najstaršie. Väčšinou však budeme mať priestoru dostatok – maximálna veľkosť sa typicky nastavuje na veľmi veľkú hodnotu takže sa skúsenosti často nevymazávajú vôbec.

Okrem maximálnej veľkosti pamäte nastavujeme aj predvolenú veľkosť dávky (`batch_size`). Keď zavoláme metódu `.sample()`, dostaneme dávku s `batch_size` náhodne vybranými stavovými prechodmi.



In [ ]:
replay_buffer = ReplayBuffer(max_size=10000, batch_size=100)

Aby sme v prehrávacej pamäti zaznamenali stavový prechod, môžeme zavolať `replay_buffer.add(obs, a, reward, obs_next, done, info)`, kde `a` je akcia a `obs` je predprechodové pozorovanie, zatiaľ čo `obs_next` je pozorovanie po stavovom prechode.

---
#### Úloha 3: Zaznamenanie prechodov

**Spustite prostredie a náhodne voľte akcie. Pomocou metódy `replay_buffer.add` zaznamenajte 1000 prechodov do prehrávacej pamäte.** 

Tipy:

* Spomeňte si, že `env.step(action)` navracia n-ticu `(observation, reward, done, info)`.
* Spomeňte si tiež, že keď `done` má hodnotu true, aktuálna epizóda skončila a treba začať novú epizódu volaním `obs = env.reset()`.
* Náhodné akcie možno z priestoru akcií vzorkovať pomocou volania `env.action_space.sample()`.
---


In [ ]:
replay_buffer = ReplayBuffer(max_size=10000, batch_size=100)

env = MazeEnv()
obs = env.reset()
done = False

for step in range(1000):
    
    
    
    # ----
    
    
    
    obs = obs_next
        if terminated or truncated: obs = env.reset()

In [ ]:
obs, a, reward, obs_next, done, info = replay_buffer.sample()

print(f"The replay buffer contains {len(replay_buffer)} samples.")
print(f"The batch contains {len(obs)} samples.\n")

print(f"obs: {obs[:3]} ...")
print(f"actions: {a[:3]} ...")
print(f"obs_next: {obs_next[:3]} ...")
print(f"done: {done[:3]} ...")
print(f"info: {info[:3]} ...")

#### Q učenie s prehrávacou pamäťou

Ďalej budeme znovu experimentovať s Q učením, pričom teraz už budeme používať aj $\varepsilon$-greedy stratégiu, aj prehrávaciu pamäť. Znovu použijeme už hotovú implementáciu danej metódy.

Čo by sme mali vidieť je, že hodnoty sa budú šíriť omnoho rýchlejšie, pretože tie isté skúsenosti sa prehrávajú znovu a znovu. Okrem toho by malo byť vidno, že sa aktualizujú aj hodnoty stavov, ktoré neboli navštívené v rámci ostatnej epizódy. Pri hodnote $\varepsilon = 0.1$ však agent stále nepreskúma adekvátne všetky oblasti stavového priestoru.



In [ ]:
seed(1)

env = MazeEnv(
    show_path=True,
    show_path_kw=dict(show_arrows=False, show_visited=True)
)

plotter = Plotter(env, ActionValueTable, 
    (StateValueTable, {'render_kwargs': {'skip': {'player_logger'}}}),
     figsize=[8, 4], render_agent=False
)

qtable = ActionValueTable(env.action_space.n)
algo = QLearning(qtable, alpha=0.5, gamma=0.9)
policy = EpsGreedyPolicy(qtable, env.action_space.n, epsilon=0.1)
replay_buffer = ReplayBuffer(max_size=10000, batch_size=100)

trainer = Trainer(
    algo, policy, verbose=5, replay_buffer=replay_buffer,
    on_end_episode=[lambda *args: plotter.plot(qtable, qtable.to_state_values())],
)

trainer.train(env, max_episodes=20, max_episode_steps=1000)

In [ ]:
env = MazeEnv(show_path=True)
qtable_control(env, qtable, max_steps=100)
env.render()

### Žíhanie miery prieskumu

Ako sme videli, ak používame prinízku hodnotu $\varepsilon$, vedie to k nedostatočnému prieskumu. Ak na druhej strane nastavíme $\varepsilon$ na privysokú hodnotu, správanie sa agenta bude nestále aj potom, ako sa priblížil ku skutočnej hodnotovej funkcii akcií. Výsledkom je, že sa môže vzdať veľmi veľkého množstva odmien, epizóda môže trvať zbytočne veľa krokov atď.

Je preto rozumné mieru prieskumu žíhať: aby agent realizoval veľa prieskumu na začiatku, ale aby sa miera prieskumu postupne znižnovala takže sa agent postupom času bude blížiť ku optimálnej stratégii, využívať a zdokonaľovať nadobudnuté poznatky.

Častým spôsobom, ako mieru prieskumu žíhať, je predpísať jej exponenciálny rozvrh. Tu na to použijeme triedu `ExponentialSchedule`, v rámci ktorej môžeme špecifikovať:

* Počiatočnú hodnotu parametra;
* Konečnú hodnotu parametra;
* Krok, kedy má začať žíhanie;
* Krok, kedy sa má žíhanie ukončiť.
V našom prípade bude žíhanie výchádzať z čísla epizódy (`method='episode'`) a nie z počtu krokov. Počet krokov by však bolo možné použiť takisto a dokonca je to pravdepodobne bežnejšia voľba. Takto môže náš exponenciálny rozvrh vyzerať:



In [ ]:
eps_schedule = ExponentialSchedule(
    None,
    init_val=1.0, final_val=0.1,
    first_step=5, final_step=18,
    method='episode'
)

steps = range(eps_schedule.final_step+5)
eps = [eps_schedule.get_value(s) for s in steps]
plt.plot(steps, eps)
plt.xlabel("episode")
plt.ylabel("epsilon")
plt.grid(ls='--')

Keď sme si rozvrh zadefinovali, zaregistrujeme ho v rámci callback-u `on_begin_step`, aby sa $\varepsilon$ podľa neho priebežne aktualizoval. Pozrime sa teraz, aký efekt to bude mať.

Keďže v rámci niekoľkých počiatočných epizód držíme $\varepsilon$ na hodnote $1$, agent sa vtedy bude správať úplne náhodným spôsobom. To by nám malo pomôcť nazbierať počas tých epizód veľa skúseností. Následne sa už bude $\varepsilon$ postupne znižovať.



In [ ]:
seed(1)

env = MazeEnv(
    show_path=True,
    show_path_kw=dict(show_arrows=False, show_visited=True)
)

plotter = Plotter(env, ActionValueTable, 
    (StateValueTable, {'render_kwargs': {'skip': {'player_logger'}}}),
     figsize=[8, 4], render_agent=False
)

qtable = ActionValueTable(env.action_space.n)
algo = QLearning(qtable, alpha=0.5, gamma=0.9)
policy = EpsGreedyPolicy(qtable, env.action_space.n)

eps_schedule = ExponentialSchedule(
    policy.set_epsilon,
    init_val=1.0, final_val=0.1,
    first_step=5, final_step=18,
    method='episode'
)

replay_buffer = ReplayBuffer(max_size=10000, batch_size=100)

trainer = Trainer(
    algo, policy, replay_buffer, verbose=5,
    on_begin_step=[eps_schedule],
    on_end_episode=[lambda *args: plotter.plot(qtable, qtable.to_state_values())],
)

trainer.train(env, max_episodes=20, max_episode_steps=1000)

In [ ]:
env = MazeEnv(show_path=True)
qtable_control(env, qtable, max_steps=100)
env.render()